In [16]:
import pandas as pd
df = pd.read_excel(r'C:\Users\LENOVO\Desktop\Python\dataset_for_deepLearning\sentiment_analysis_for_financial_news\all-data.xlsx', engine='openpyxl')
df.dropna(inplace = True)
df.shape

(4846, 2)

In [17]:
df.head()

,label,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [18]:
etiket=df.iloc[:,0:1].values
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
etiket[:,0]=le.fit_transform(df.iloc[:,0])
ohe=preprocessing.OneHotEncoder()
etiket=ohe.fit_transform(etiket).toarray()

In [19]:
X=df.iloc[:,1:2].values
print(type(X))
print(type(etiket))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, etiket, test_size=0.2, random_state=42)

In [23]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3876, 1)
(3876, 3)
(970, 1)
(970, 3)


In [24]:
texts=[]
for x in range(0,X_train.shape[0]):
    texts.append(X_train[x,0])

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
maxlen=100
training_samples=3000
validation_samples=876
max_words=10000
tokenizer=Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts)
word_index=tokenizer.word_index
print("Found %s unique tokens."%len(word_index))
data=pad_sequences(sequences,maxlen=maxlen)
labels=np.asarray(y_train)
print("Shape of data tensor:",data.shape)
print("Shape of label tensor:",labels.shape)
indices=np.arange(data.shape[0])
np.random.shuffle(indices)
data=data[indices]
labels=labels[indices]
x_train=data[:training_samples]
y_train=labels[:training_samples]
x_val=data[training_samples:training_samples+validation_samples]
y_val=labels[training_samples:training_samples+validation_samples]

Found 9001 unique tokens.
Shape of data tensor: (3876, 100)
Shape of label tensor: (3876, 3)


In [26]:
import os
glove_dir="C:/Users/LENOVO/Desktop/Python/dataset_for_deepLearning/Imdb/glove.6B"
embeddings_index={}
f=open(os.path.join(glove_dir,"glove.6B.200d.txt"),encoding="utf8")
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype="float32")
    embeddings_index[word]=coefs
f.close()
print("found %s word vectors"%len(embeddings_index))

found 400000 word vectors


In [27]:
embedding_dim=200
embedding_matrix=np.zeros((max_words,embedding_dim))
for word, i in word_index.items():
    if i<max_words:
        embedding_vector=embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector

In [28]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, SimpleRNN, LSTM, Bidirectional, GRU
model=Sequential()
model.add(Embedding(max_words,embedding_dim,input_length=maxlen))
model.add(Bidirectional(GRU(32,dropout=0.5,recurrent_dropout=0.5,return_sequences=True)))
model.add(Bidirectional(GRU(64,activation='relu',dropout=0.5,recurrent_dropout=0.5)))
model.add(Dense(3,activation="softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 200)          2000000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100, 64)           44736     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 128)               49536     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 2,094,659
Trainable params: 2,094,659
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable=False

In [30]:
from keras.callbacks import ModelCheckpoint
model_path="C:/Users/LENOVO/Desktop/Python/dataset_for_deepLearning/sentiment_analysis_for_financial_news/saved_models/weights.{epoch:02d}-{val_acc:.2f}.h5"
mc =ModelCheckpoint(filepath=model_path, monitor='val_acc',verbose=1, mode="max", save_best_only=True)
callback_list=[mc]

In [31]:
model.compile(optimizer="rmsprop",
             loss="categorical_crossentropy",
             metrics=["acc"])
history=model.fit(x_train,y_train,
                 epochs=40,
                 batch_size=32,
                 callbacks=callback_list,
                 validation_data=(x_val,y_val))

Train on 3000 samples, validate on 876 samples
Epoch 1/40
3000/3000 [==============================] - 39s - loss: 0.9249 - acc: 0.5970 - val_loss: 0.8879 - val_acc: 0.5833
Epoch 2/40
3000/3000 [==============================] - 28s - loss: 0.8746 - acc: 0.6077 - val_loss: 0.8659 - val_acc: 0.6358
Epoch 3/40
3000/3000 [==============================] - 27s - loss: 0.8611 - acc: 0.6177 - val_loss: 0.8354 - val_acc: 0.6279
Epoch 4/40
3000/3000 [==============================] - 27s - loss: 0.8360 - acc: 0.6347 - val_loss: 0.7978 - val_acc: 0.6495
Epoch 5/40
3000/3000 [==============================] - 30s - loss: 0.8088 - acc: 0.6430 - val_loss: 0.7788 - val_acc: 0.6427
Epoch 6/40
3000/3000 [==============================] - 30s - loss: 0.7965 - acc: 0.6540 - val_loss: 0.7590 - val_acc: 0.6450
Epoch 7/40
3000/3000 [==============================] - 25s - loss: 0.7780 - acc: 0.6613 - val_loss: 0.7771 - val_acc: 0.6495
Epoch 8/40
3000/3000 [==============================] - 23s - loss: 0.7

3000/3000 [==============================] - 30s - loss: 0.5837 - acc: 0.7560 - val_loss: 0.5483 - val_acc: 0.7683
Epoch 26/40
3000/3000 [==============================] - 26s - loss: 0.5633 - acc: 0.7690 - val_loss: 0.5439 - val_acc: 0.7717
Epoch 27/40
3000/3000 [==============================] - 25s - loss: 0.5454 - acc: 0.7760 - val_loss: 0.5466 - val_acc: 0.7705
Epoch 28/40
3000/3000 [==============================] - 29s - loss: 0.5392 - acc: 0.7753 - val_loss: 0.5186 - val_acc: 0.7934
Epoch 29/40
3000/3000 [==============================] - 27s - loss: 0.5343 - acc: 0.7727 - val_loss: 0.5358 - val_acc: 0.7740
Epoch 30/40
3000/3000 [==============================] - 28s - loss: 0.5341 - acc: 0.7837 - val_loss: 0.5159 - val_acc: 0.7945
Epoch 31/40
3000/3000 [==============================] - 26s - loss: 0.5101 - acc: 0.7993 - val_loss: 0.5133 - val_acc: 0.7945
Epoch 32/40
3000/3000 [==============================] - 32s - loss: 0.5055 - acc: 0.7950 - val_loss: 0.5064 - val_acc: 0.7

In [32]:
test_texts=[]
for x in range(0,X_test.shape[0]):
    test_texts.append(X_test[x,0])

In [33]:
sequences=tokenizer.texts_to_sequences(test_texts)
x_test=pad_sequences(sequences,maxlen=maxlen)
y_test=np.asarray(y_test)

In [34]:
from keras.models import load_model
new_model=load_model("C:/Users/LENOVO/Desktop/Python/dataset_for_deepLearning/sentiment_analysis_for_financial_news/saved_models/weights.34-0.80.h5")

In [35]:
test_loss, test_acc=new_model.evaluate(x_test,y_test)
print('test acc:', test_acc)
print('test loss:',test_loss)

960/970 [============================>.] - ETA: 0stest acc: 0.8030927835051547
test loss: 0.5087852184305486
